In [ ]:
 
# Import Langchain dependencies
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import Streamlit for UI development
import streamlit as st

# Import WatsonX Langchain interface
from watsonxlangchain import LangChainInterface

# Set your WatsonX credentials
creds = {
    'apikey': 'ZNPRw05jup0f2SFOCFkAtfZskeCPL6fESGaYAq1EWRp7',
    'url': 'https://us-south.ml.cloud.ibm.com'
}

# Create LLM using Langchain
llm = LangChainInterface(
    credentials=creds,
    model='meta-llama/llama-2-70b-chat',
    params={
        'decoding_method': 'sample',
        'max_new_tokens': 200,
        'temperature': 0.5
    },
    project_id='dd58941e-28cc-4abe-9189-0bebc2f2edec'
)

# Title of the app
st.title('Ask WatsonX')

# Setup session state to store messages
if 'messages' not in st.session_state:
    st.session_state.messages = []

# Prompt input box
prompt = st.chat_input('Pass Your Prompt here')

# Display previous messages
for message in st.session_state.messages:
    st.chat_message(message['role']).markdown(message['content'])

# Handle user input
if prompt:
    st.chat_message('user').markdown(prompt)
    st.session_state.messages.append({'role': 'user', 'content': prompt})
    response = llm(prompt)
    st.chat_message('assistant').markdown(response)
    st.session_state.messages.append({'role': 'assistant', 'content': response})

# Cache and load the PDF vector index
@st.cache_resource
def load_pdf():
    pdf_name = 'what is generative ai.pdf'  # Update if needed
    loaders = [PyPDFLoader(pdf_name)]
    index = VectorstoreIndexCreator(
        embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2'),
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
    ).from_loaders(loaders)
    return index

# Load vector index
index = load_pdf()

# Create Q&A retrieval chain from vectorstore
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=index.vectorstore.as_retriever()
)

# Optional: Enable document-based Q&A
doc_question = st.text_input("Ask a question about the PDF document")
if doc_question:
    doc_answer = chain.run(doc_question)
    st.write("Answer:", doc_answer)
